In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

data_path = '/content/drive/MyDrive/training'
class_names = os.listdir(data_path)
num_classes = len(class_names)

data = []
labels = []

for class_name in class_names:
    class_path = os.path.join(data_path, class_name)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Invalid image file: {image_path}")
            image = cv2.resize(image, (224, 224))
            data.append(image)
            labels.append(class_names.index(class_name))
        except Exception as e:
            print(f"Error processing image: {image_path}")
            print(e)

data = np.array(data)
labels = np.array(labels)



Splitting the dataset into Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

mean = np.mean(train_data, axis=(0, 1, 2))
std = np.std(train_data, axis=(0, 1, 2))

train_data = (train_data - mean) / std
test_data = (test_data - mean) / std


Installing the required packages

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))


convnext model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import convnext
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras import regularizers
import numpy as np

# Load the pre-trained ConvNeXtBase model without the top layers
base_model = convnext.ConvNeXtBase(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)  # L2 Regularization
x = BatchNormalization()(x)  # Add Batch Normalization
x = Dropout(0.5)(x)  # Dropout to reduce overfitting
predictions = Dense(num_classes, activation='softmax')(x)  # Softmax activation for multi-class classification

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers for initial training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

# Callbacks for learning rate reduction, early stopping, and learning rate scheduling
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.2, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
history = model.fit(
    train_data,
    train_labels,
    epochs=20,
    batch_size=32,
    validation_data=(test_data, test_labels),
    callbacks=[lr_reduction, early_stopping, lr_scheduler]
)

# Fine-tuning: Unfreeze base model layers and continue training
for layer in base_model.layers:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Continue training (fine-tuning)
history_fine_tune = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(test_data, test_labels),
    callbacks=[lr_reduction, early_stopping, lr_scheduler]
)

# Evaluate the model on validation data
loss, accuracy = model.evaluate(test_data, test_labels)
print("train Loss:", loss)
print("train Accuracy:", accuracy)

# Save the model for later use
model.save('/content/retinal_disease_model_convnext.h5')


350926856/350926856 ━━━━━━━━━━━━━━━━━━━━ 18s 0us/step
Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.5254 - loss: 2.5558 - val_accuracy: 0.4907 - val_loss: 2.9111 - learning_rate: 0.0010
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 422ms/step - accuracy: 0.8597 - loss: 1.3667 - val_accuracy: 0.4348 - val_loss: 2.4167 - learning_rate: 0.0010
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 382ms/step - accuracy: 0.9184 - loss: 1.1526 - val_accuracy: 0.5652 - val_loss: 2.0113 - learning_rate: 0.0010
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 449ms/step - accuracy: 0.9286 - loss: 1.0680 - val_accuracy: 0.7267 - val_loss: 1.6840 - learning_rate: 0.0010
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 431ms/step - accuracy: 0.9578 - loss: 0.9817 - val_accuracy: 0.7888 - val_loss: 1.3993 - learning_rate: 0.0010
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 430ms/step - accuracy: 0.9538 - loss: 0.9623 - val_accuracy: 0.7826 - val_loss: 1.3340 - learning_rate: 0.0010
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━

Test Loss: 0.6485193371772766
Test Accuracy: 0.9937888383865356
